In [ ]:
from lockin import lockinAmp
import numpy as np
import time
import pickle as pkl
from qcodes.instrument_drivers.BlueFors.BlueFors import BlueFors

% autoreload 2

In [ ]:
data_dir = '/path/to/data/folder'
BF_log_dir = '/path/to/log/folder'

In [ ]:
# Create an instance of the BlueFors instrument
bluefors = BlueFors(name='bluefors',
                    folder_path=BF_log_dir,
                    channel_vacuum_can=1,
                    channel_pumping_line=2,
                    channel_compressor_outlet=3,
                    channel_compressor_inlet=4,
                    channel_mixture_tank=5,
                    channel_venting_line=6,
                    channel_50k_plate=7,
                    channel_4k_plate=8,
                    channel_still=9,
                    channel_mixing_chamber=10,
                    channel_magnet=11)

# Connect to the BlueFors instrument
bluefors.connect_message()

# Connect to the lock-in amplifier
SR_DSP7265 = lockinAmp(GPIBAddress=12)

In [ ]:
# Set time constant and sensitivity

# sensitivity range: 2nV to 1V
# time constant range: 10us to 100ks

# The lock-in set point will be rounded up to the nearest value in the lock-in's list of available values.

SR_DSP7265.set_time_constant(1) # seconds
SR_DSP7265.set_sensitivity(0.5) # volts

In [ ]:
# Initialize scan parameters

scan_pt_num = 1000
delay = 10 # seconds

min_freq = 10 # Hz
max_freq = 10e3 # Hz
freq_num = 100
freqs = np.linspace(min_freq, max_freq, freq_num)

amp_out = 1

data = np.zeros((scan_pt_num, 4, freq_num))

In [ ]:
# Perform the scan

for i in range(scan_pt_num):

    temperature_MXC = bluefors.temperature_mixing_chamber() # in Kelvin. I actually don't know the function name, so I just made it up.
    amp, phase = SR_DSP7265.freq_scan(amp_out, freqs)
    data[i, 0, :] = temperature_MXC*np.ones(freq_num)
    data[i, 1, :] = freqs
    data[i, 2, :] = amp
    data[i, 3, :] = phase

    time.sleep(delay)

In [ ]:
# Create a dictionary to save the data

data_dict = {'temperature': data[:, 0, :],
                'frequency': data[:, 1, :],
                'amplitude': data[:, 2, :],
                'phase': data[:, 3, :]}

# Save the data
pkl.dump(data_dict, open(data_dir + 'data.pkl', 'wb'))


In [ ]:
# Disconnect from the BlueFors instrument
bluefors.disconnect()